In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os
tf.__version__

'2.0.0'

In [2]:
train_image_path = glob.glob('./dc_2000/train/*/*.jpg')
len(train_image_path)

2000

In [3]:
train_image_path[-5:]

['./dc_2000/train\\dog\\dog.995.jpg',
 './dc_2000/train\\dog\\dog.996.jpg',
 './dc_2000/train\\dog\\dog.997.jpg',
 './dc_2000/train\\dog\\dog.998.jpg',
 './dc_2000/train\\dog\\dog.999.jpg']

In [4]:
p = './dc_2000/train\\dog\\dog.995.jpg'

In [5]:
int(p.split('\\')[1] == 'dog')

1

In [6]:
train_image_label = [int(p.split('\\')[1] == 'cat') for p in train_image_path]
train_image_label[-5:]

[0, 0, 0, 0, 0]

In [7]:
#数据处理管道
def load_and_preprosess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)   
    image = tf.image.resize(image, [256,256])
    image = tf.cast(image, tf.float32)
    image = image/255
    label = tf.reshape(label, [1])
    return image, label

In [8]:

def load_and_preprosess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)   
    image = tf.image.resize(image, [360,360])
    
    #图片增强
    image = tf.image.random_crop(image, [256, 256, 3])                      #随机裁剪
    image = tf.image.random_flip_left_right(image)                               #随机左右翻转
    image = tf.image.random_flip_up_down(image)                                  #随机上下翻转
    image = tf.image.random_brightness(image, 0.5)                          #随机亮度
    image = tf.image.random_contrast(image, 0,1)                            #对比度
    
    image = tf.cast(image, tf.float32)
    image = image/255
    label = tf.reshape(label, [1])
    return image, label




In [9]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path, train_image_label))   #包含路径和label的dataset

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE                     #AUTOTUNE根据cpu个数自动实现并行运算

In [11]:
train_image_ds = train_image_ds.map(load_and_preprosess_image, num_parallel_calls=AUTOTUNE)           #map函数，映射到所有对象上,并行运算个数设置为AUTOTUNE

In [12]:
train_image_ds

<ParallelMapDataset shapes: ((256, 256, 3), (1,)), types: (tf.float32, tf.int32)>

In [13]:
BATCH_SIZE = 32
train_count = len(train_image_path)

In [14]:
train_image_ds = train_image_ds.shuffle(train_count).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)                   #在训练同时在后台预读取，加快训练速度

In [15]:
imgs, labels = next(iter(train_image_ds))

In [16]:
imgs.shape

TensorShape([32, 256, 256, 3])

In [ ]:
plt.imshow(imgs[0])

In [18]:
labels[0]

<tf.Tensor: id=127, shape=(1,), dtype=int32, numpy=array([0])>

test数据集准备过程

In [19]:
test_image_path = glob.glob('./dc_2000/test/*/*.jpg')
test_image_label = [int(p.split('\\')[1] == 'cat') for p in test_image_path]
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path, test_image_label))
test_image_ds =test_image_ds.map(load_and_preprosess_image, num_parallel_calls=AUTOTUNE)
test_image_ds = train_image_ds.batch(BATCH_SIZE)
test_image_ds = train_image_ds.prefetch(AUTOTUNE)

创建模型

In [20]:
model =  keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), input_shape = (256, 256, 3), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512, (3,3), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(1024, (3,3), activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 254, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 125, 128)     512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 256)       2

In [22]:
pred = model(imgs)
pred.shape

TensorShape([32, 1])

In [23]:
np.array([p[0].numpy() for p in tf.cast(pred > 0, tf.int32)])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [24]:
np.array([l[0].numpy() for l in labels])

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0])

In [25]:
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)    #大写的是可调用方法

In [26]:
loss_func([0., 0., 1., 1.], [1., 1., 1., 1.])    #计算交叉熵，第一个数组是真实值，第二组是预测值

<tf.Tensor: id=1623, shape=(), dtype=float32, numpy=0.81326175>

In [27]:
loss_func([[0.], [0.], [1.], [1.]], [[1.], [1.], [1.], [1.]])

<tf.Tensor: id=1655, shape=(), dtype=float32, numpy=0.81326175>

In [28]:
tf.keras.losses.binary_crossentropy([0., 0., 1., 1.], [1., 1., 1., 1.])   #大写的是可调用方法，小写的需要把计算数值写进去

<tf.Tensor: id=1679, shape=(), dtype=float32, numpy=7.6666193>

In [29]:
optimizer = tf.keras.optimizers.Adam()

In [30]:
epoch_loss_avg = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

epoch_loss_avg_test = tf.keras.metrics.Mean('train_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [31]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [32]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)   #计算pre和labels之间的二元交叉熵损失
        
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    epoch_loss_avg(loss_step)
    train_accuracy(labels, pred)

In [33]:
def test_step(model, images, labels):
    pred = model(images, training = False)
    loss_step = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, pred)
    
    epoch_loss_avg_test(loss_step)
    test_accuracy(labels, tf.cast(pred>0.5, tf.int32))

In [34]:
train_loss_result = []
train_acc_resulet = []

test_loss_result = []
test_acc_resulet = []

In [35]:
def train():
    num_epochs = 30
    for ele in range(num_epochs):
        for images_,labels_ in train_image_ds: 
            train_step(model, images_, labels_)
            print('.', end='')
        print()
        
        train_loss_result.append(epoch_loss_avg.result())
        train_acc_resulet.append(train_accuracy.result())
        
        for images_,labels_ in test_image_ds: 
            test_step(model, images_, labels_)
            print('.', end='')
        print()
            
        test_loss_result.append(epoch_loss_avg_test.result())
        test_acc_resulet.append(test_accuracy.result())
        
        print('Epoch:{}, loss:{:.3f}, accuracy:{:.3f}, test_loss:{:.3f}, test_accuracy:{:.3f}'.format(
            ele+1, epoch_loss_avg.result(), train_accuracy.result(), epoch_loss_avg_test.result(), test_accuracy.result()))
        
        epoch_loss_avg.reset_states()
        train_accuracy.reset_states()
        epoch_loss_avg_test.reset_states()
        test_accuracy.reset_states()        

In [36]:
train()                                                       #

...........................

KeyboardInterrupt: 